Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## House Prices Prediction
### Model Deployment to Azure Container Instances (ACI) and Azure Kubernetes Services (AKS)

We finished the last Notebook by finding best fitting model using AutoML and registering it to our AML account. In this Notebook, we deploy this model to an ACI instance and test it by scoring in real-time.

We begin by importing the necessary packages and setting some notebook options.

In [ ]:
import json
import logging
import seaborn as sns
import warnings
import os, logging
import random
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from pprint import pprint

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

from azureml.core.image import Image, ContainerImage
from azureml.core.model import Model
from azureml.core.webservice import Webservice

image_name = "house-prices-img"
aci_service_name = "house-prices-aci"

warnings.filterwarnings('ignore')

%matplotlib inline

pd.options.display.max_rows = None
pd.options.display.max_columns = None

Now we instantiate a [Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspaces) object, using the information from the configuration file that we uploaded previously.

In [ ]:
ws = Workspace.from_config(path="./config/ws_config.json")

The first part of the deployment consists of pointing to the model we want to deploy. We can simply provide the model name, which was given to us at the time we registered the model (in a previous notebook). The easiest thing for us is to go to the Azure portal to look up the model name: from the AML Workspace page, click on the section called **Models**. Once you find the model, copy its name. You will need to paste it in two separate cells below.

Here's a quick sanity check to ensure that the model exists and can be loaded (loading the model in the current session is not required for deployment). Copy and paste the model name from the previous notebook in the below cell.

In [ ]:
from azureml.core.model import Model
model_name = "" # PASTE MODEL NAME HERE

model = Model(workspace=ws, name=model_name)
print(model.id)

We now create a scoring script that will run every time we make a call to the deployed model. The scoring script consists of an `init` function that will load the model and a `run` function that will load the data we provide at score time and use the model to obtain predictions.

In [ ]:
%%writefile score.py
import pickle
import json
import numpy
import pandas as pd
from sklearn.externals import joblib
from azureml.core.model import Model
import azureml.train.automl

model_name = "" # PASTE MODEL NAME HERE

def init():
    global model
    model_path = Model.get_model_path(model_name = model_name)
    model = joblib.load(model_path) # deserialize the model file back into a sklearn model

def run(rawdata):
    try:
        data = json.loads(rawdata)['data']
        data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()})

Next we create a `yml` file for the conda environment that will serve as runtime for the scoring script above. To ensure consistency of the scored results with the training results, the dependencies need to mirror development environment (used for model training), but pared down to what is needed for scoring.

In [ ]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - scikit-learn==0.19.1
    - azureml-sdk[automl]==1.0.65

Using the scoring script and conda environment file, we can now create a docker image that will host the scoring script and a Python executable that meets the conda requirement dependencies laid out in the YAML file.

In [ ]:
image_config = ContainerImage.image_configuration(runtime="python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'area': "housing", 'type': "automl_classification"},
                                 description = "Image for housing price prediction using AutoML")

From the image config file above we now create a Docker image.

In [ ]:
%%time
image = Image.create(name = image_name,
                     models = [model], 
                     image_config = image_config, 
                     workspace = ws)

image.wait_for_creation(show_output = True)

If the image creation fails, this is how we can access the log file and examine what went wrong.

In [ ]:
print(image.image_build_log_uri)

Here is the image location that will be used when the imaged is pulled down. Creating a new image with the same name will result in a new version of the image.

In [ ]:
print(image.image_location)

Note that if the image was created in another session and we just wanted to point to it in this session, then we can just pass the image name and workspace to the `Image` function as follows:

In [ ]:
image = Image(name = image_name, workspace = ws)
print(image.image_location)

We are now ready to deploy our image as a web service on ACI. To do so, we first create a config file and then pass it to `deploy_from_image` along with a name for the service, the image we created in the last step, and our AML Workspace.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"method" : "automl"}, 
                                               description='Predictive maintenance using auto-ml')

If a service with the same name already exists, we can delete it by calling the `delete` method.

In [ ]:
# aci_service.delete()

In [ ]:
%%time
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)

In case the service creation fails we can uncomment and the next cell to check out the logs.

In [ ]:
# pprint(aci_service.get_logs())

It is time to test our web service. To begin with, we will point to our service using `Webservice`. Note that we've already done this in the last step, so in the current session this is not a necessary step, but since we want to be able to test the service from any Python session, we will point to the service again here. There is next to no overhead in doing so.

In [ ]:
aci_service = Webservice(workspace = ws, name = aci_service_name)

We can now proceed to testing the service. To do so, we have a sample json file with some data in it. This will act as the data that we use for scoring.

In [ ]:
test_samples = open("sample_data.json", encoding = 'utf8').read()
print(test_samples)

We can pass this data to the service using the `run` method, and it will return the predictions to us.

In [ ]:
prediction = aci_service.run(input_data = test_samples)
print(prediction)

If you have Postman or any other REST client you can also test the scoring service against it.

If we want to test the scoring service locally prior to deployment, here's how: We first read the json file into a Pandas data frame.

In [ ]:
data = json.loads(test_samples)['data']
data = pd.DataFrame.from_dict(data)
data

We now download the model we registered under our AML Workspace as a pickle file and load it into the current python session.

In [ ]:
model.download(exist_ok=True)
model_path="model.pkl"
mod = joblib.load(model_path)

Finally, we run the model's `predict` method to get predictions and return them as a json object.

In [ ]:
result = mod.predict(data)
json.dumps({"result":result.tolist()})

The above three steps are the same ones happening in the scoring script itself.